# BUNDESDATA RAG 


In [32]:
import json, requests
from typing import Any, List 
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_ollama import OllamaEmbeddings, ChatOllama
from langchain_community.vectorstores import Chroma
from ollama._types import ResponseError

## Daten laden und vorbereiten 

In [33]:
BASEURL = "https://verkehr.autobahn.de/o/autobahn"
SERVICES = [
    "roadworks",
    "parking_lorry",
    "warning",
    "closure",
    "electric_charging_station",
]

**stringify_all_fields:**
Conversion of a python object (dict) into string required for document indexing and processing using langchain.

In [34]:
def stringify_all_fields(obj: Any) -> str:
    return json.dumps(obj, ensure_ascii=False, indent=2, sort_keys=True) #ensure ascii=False to handle special characters

In [35]:
def get_items(payload: Any, service: str) -> List[Any]:
    # if payload is a dict with the service as a key and the value is a list, return that list
    if isinstance(payload, dict) and isinstance(payload.get(service), list):
        return payload[service]
    # if payload is already a list, return it directly
    if isinstance(payload, list):
        return payload
    return [payload]

**to_documents:**
Helper to convert the json response (dict) into langchain documents using the stringify_all_fields() function to get a text representation of all fields.
Additionally, we capture data about the road_id, service, title, description to give unique markers to roads.

In [36]:
def to_documents(road_id: str, service: str, items: List[Any], source_url: str) -> List[Document]:
    docs: List[Document] = [] # initialize empty list
    for i, item in enumerate(items):
        # define title and description
        title = ""
        description = ""
        if isinstance(item, dict):
            title = item.get("title", "") or item.get("name", "") # if title is empty, try name
            description = item.get("description", "") or item.get("text", "") # if description is empty, try text

        page_content = "\n".join(
            # structured content
            [
                f"road: {road_id}",
                f"service: {service}",
                (f"title: {title}" if title else ""),
                (f"description: {description}" if description else ""),
                "",
                "ALL_FIELDS_JSON:",
                # use stringify_all_fields to convert item to string
                stringify_all_fields(item),
            ]
        ).strip()
        # create document and add to list
        docs.append(
            Document(
                page_content=page_content,
                metadata={
                    "road_id": road_id,
                    "service": service,
                    "source_url": source_url,
                    "item_index": i,
                    "title": title,
                },
            )
        )
    return docs

**build_documents():**
This function does the API calls. We go through all road ids and services (endpoints) available and convert them into a list of langchain documents.

In [37]:
def build_documents() -> List[Document]:
    docs: List[Document] = [] # initialize empty list
    with requests.Session() as s:
        # get list of roads
        roads_resp = s.get(BASEURL, timeout=20)
        roads_resp.raise_for_status()
        roads = roads_resp.json()["roads"]
        # iterate over roads and services
        for road_id in roads:
            for service in SERVICES:
                # fetch service data for every road
                url = f"{BASEURL}/{road_id}/services/{service}"
                resp = s.get(url, timeout=20)
                resp.raise_for_status()
                # get payload
                payload = resp.json()
                # extract items with helper function
                items = get_items(payload, service)
                docs.extend(to_documents(road_id, service, items, url)) # convert to documents with helper function
    return docs

## Embedding Model initialisieren (mit dem Uni-Ollama-Server)

In [38]:
LLM_URL = "http://132.199.138.16:11434"
LLM_MODEL = "gpt-oss:20b"
EMBEDDING_MODEL = "nomic-embed-text"

**Split text using RecursiveCharacterTextSplitter**

In [39]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=512,
    chunk_overlap=50,
    separators=["\n", ",", " ", ""]
)

docs = build_documents()

splitted_docs = text_splitter.split_documents(docs)

print(len(splitted_docs))

29121


**add_in_batches():**

We encountered a EOF status code: 500 Error during the embedding request. To fix this, we batched the documents in even smaller batches (batch_size = 64 here) to avoid the error.

In [40]:
def add_in_batches(
    vectorstore: Chroma,
    docs: List[Document],
    batch_size: int = 64,
    max_retries: int = 3,
):
    
    total = len(docs)
    for start in range(0, total, batch_size):
        batch = docs[start : start + batch_size]

        for attempt in range(1, max_retries + 1):
            try:
                vectorstore.add_documents(batch)
                break
            except ResponseError as e:
                if attempt == max_retries:
                    raise

**Initialize Ollama-Embedding-Model**

In [41]:
ollama_embeddings = OllamaEmbeddings(
    model=EMBEDDING_MODEL,
    base_url=LLM_URL
)

try:
    vectorstore.delete_collection()

except:
    pass


vectorstore = Chroma(collection_name="autobahn-rag", embedding_function=ollama_embeddings)

add_in_batches(
    vectorstore,
    splitted_docs,
    batch_size = 32,
    max_retries=3
)

print("Docs in Chroma:", vectorstore._collection.count())

vector_retriever = vectorstore.as_retriever(search_kwargs={"k":5})

Docs in Chroma: 29121


In [42]:
llm = ChatOllama(
    model=LLM_MODEL,
    base_url=LLM_URL,
    temperature=1
)

In [43]:
question = "Parkplatz auf der A3"
context_docs = vector_retriever.invoke(question)
context = "\n\n---\n\n".join(d.page_content for d in context_docs)

prompt = ("Beantworte die Frage mit Kontext."
          "Wenn Informationen fehlen, sage, was im Kontext nicht enthalten ist, aber antworte trotzdem so weit wie möglich.\n\n"
          f"Frage: {question} \n\n Kontext:{context}"
)

answer = llm.invoke(prompt)
print("\nQuestion:", question)
print("\n[ANSWER]:", answer.content)


Question: Parkplatz auf der A3

[ANSWER]: **Antwort:**

Im bereitgestellten Kontext sind keine konkreten Angaben zu Parkplätzen auf der A3 vorhanden. Die vorliegenden Informationen beschreiben lediglich Bauarbeiten und Verkehre auf der A3 (z. B. Umbau am AK Kaiserberg, A3 | Köln‑Ost – Königsforst, A3 | Arnheim). 

**Was im Kontext fehlt:**

- Keine Erwähnung von Parkplätzen, Rastplätzen oder Tankstellen, die speziell auf der A3 liegen.
- Keine Angaben zu Baustellen, die Parkflächen schaffen oder einschränken könnten.
- Kein Hinweis auf besondere Parkmöglichkeiten für Baustellenbesucher oder Anwohner.

**Allgemeine Information zu Parkplätzen auf der A3:**

- Auf deutschen Autobahnen (inkl. der A3) ist dauerhaftes Parken grundsätzlich nicht erlaubt.  
- Nur an offiziellen Rast- und Tankstellen gibt es zeitlich begrenzte Parkplätze, die in der Regel für kurze Aufenthalte (z. B. 30 – 60 Minuten) gedacht sind.  
- Für Baustellenbesucher können spezielle Arbeits- oder Anwohnerparkplätze ein

## Gradio UI 

In [ ]:
import gradio as gr

def rag_answer(question: str):
    # retrieve relevant docs 
    context_docs = vector_retriever.invoke(question)

    # docs to string
    context = "\n\n---\n\n".join(d.page_content for d in context_docs)

    # promt for LLM to generate answer based on context
    prompt = (
        "Beantworte die Frage mit Kontext. "
        "Wenn Informationen fehlen, sage, was im Kontext nicht enthalten ist, aber antworte trotzdem so weit wie möglich.\n\n"
        f"Frage: {question}\n\n"
        f"Kontext:\n{context}"
    )

    # LLM call
    answer = llm.invoke(prompt)

    return answer.content


with gr.Blocks(title="Bundesdata RAG") as demo:
    gr.Markdown("# 🚗 Autobahn Bundesdata RAG")
    # instruction for user
    gr.Markdown(
        "Stelle eine Frage zum Thema deutsche Autobahnen. "
        "Es wird Kontext via Retriever geholt und das LLM antwortet soweit möglich aus dem Kontext. "
        "Im Kontext vorhandene Themenbereiche sind roadworks, parkinglorries, warnings, closures und electric charging stations."
    )

    question_in = gr.Textbox(
        label="Stelle deine Frage",
        placeholder="z.B. gib mir alle Warnings auf der A93",
        lines=2,
    )

    ask_btn = gr.Button("Antwort generieren")

    answer_out = gr.Textbox(
        label="Antwort",
        lines=10,
        interactive=False
    )

   
    # button click to generate answer
    ask_btn.click(
        fn=rag_answer,
        inputs=question_in,
        outputs=[answer_out],
    )

demo.launch()


* Running on local URL:  http://127.0.0.1:7863
* To create a public link, set `share=True` in `launch()`.
